In [1]:
import numpy as np
import scipy as sp
import scipy.constants
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
import sys
import os
import random
import glob

import subprocess as subp
import matplotlib.patches as pat
from operator import itemgetter
from scipy.optimize import minimize
from scipy.special import kn
from scipy.interpolate import interp1d
from scipy.interpolate import SmoothBivariateSpline
from scipy.optimize import brentq
from scipy.integrate import quad
from scipy.interpolate import griddata
from scipy.optimize import minimize_scalar
from scipy.optimize import root_scalar
#From http://pdg.lbl.gov/2009/AtomicNuclearProperties/HTML_PAGES/013.html
rad_length_Al=0.08897
X_Al=24.3
N_A=6.02214e23
A_Al=26.98
rad_length_coef=X_Al*N_A/A_Al
pb_to_cm2=10**-36
#Units
gev=1;mev=1e-3*gev;kev=1e-6*gev;

#constants
hbar=float(1.054*1e-34/(1.6e-19)/(1e9));
speed_of_light=scipy.constants.c;conversion=hbar**2*speed_of_light**2;
alpha_em=1.0/137.035999074;

In [2]:
def momvec(arr):
    return [arr[1],arr[2],arr[3]]
def startvec(arr):
    return [arr[5],arr[6],arr[7]]
def decaypoint(arr):
    return math.sqrt(arr[9]**2+arr[10]**2+arr[11]**2)
def dp(a1,a2):
    tot=0
    for i in range(len(a1)):
        tot+=a1[i]*a2[i]
    return tot
def veclen(a1):
    return dp(a1,a1)**0.5
def sub(a1,a2):
    return [a1[i]-a2[i] for i in range(len(a1))]
def add(a1,a2):
    return [a1[i]+a2[i] for i in range(len(a1))]
def mul(c,a1):
    return [c*a1[i] for i in range(len(a1))]
def plane_cross(p,o,r,n):
    bot=dp(n,p)
    if bot == 0:
        return -1
    a = dp(n,sub(r,o))/bot
    point = add(o,mul(a,p))
    R=sub(r,point)
    return point,dp(R,R)**0.5

In [3]:
def update_param(b,replacement_array):
    for i in range(len(b)):
        line = b[i].split()
        if len(line)<2:
            continue
        for rep in replacement_array:
            if line[0]==rep[0]:
                b[i]=line[0]+' '+str(rep[1])+'\n'
    return b
def record_list(outfile,data):
    with open(outfile,'w') as w:
        for line in data:
            tmpstr=""
            for elem in line:
                tmpstr+=str(elem)+' '
            tmpstr=tmpstr[:-1]
            tmpstr+='\n'
            w.write(tmpstr)

In [4]:
def find_total(summary_file,run_num):
    with open(summary_file) as sf:
        dats=sf.read().splitlines()
        for i in range(len(dats)):
            line=dats[i].split()
            if len(line)==2 and line[1]==run_num:
                for j in range(len(dats)-i-1):
                    line2 = dats[i+j+1].split()
                    if len(line2) > 4 and line2[0]=="Total":
                        return float(line2[3])
        print("Could not find run " + run_num)
        return -1

def find_summary(summary_file,run_num):
    with open(summary_file) as sf:
        dats=sf.read().splitlines()
        for i in range(len(dats)):
            line=dats[i].split()
            if len(line)==2 and line[1]==run_num:
                for j in range(len(dats)-i-1):
                    line2 = dats[i+j+1].split()
                    if len(line2) > 4 and line2[0]=="Total":
                        return float(line2[2]), float(line2[1]), float(line2[5]), float(line2[7]) 
        print("Could not find run " + run_num)
        return -1
    
def mom(arr):
    return math.sqrt(arr[0]**2+arr[1]**2+arr[2]**2)
    
def theta(arr):
    return math.acos(arr[2]/mom(arr))

def cos_theta(arr):
    return arr[2]/mom(arr)

def theta_dif(arr1,arr2):
    return math.acos(dp(arr1[:3],arr2[:3])/mom(arr1)/mom(arr2))

#Expect each element of arr to be 4 elements. Expects px py pz E.
def invariant_mass(arr):
    tot = [0,0,0,0]
    for line in arr:
        for i in range(4):
            tot[i]+=line[i]
    return tot[3]**2-tot[0]**2-tot[1]**2-tot[2]**2

In [5]:
hbar = 6.582e-25; c=2.99792458e8; hbc=hbar*c;
def Width_A_to_a_gamma(Ga,ma,mA):
    if(mA<ma):
        return 0
    return Ga**2*(-ma**2+mA**2)**3/(96*math.pi*mA**3)
def ldec(Gam,E_part,m_part):
    return hbc/Gam*E_part/m_part
def esc_prob(ldec,L_esc):
    return math.exp(-L_esc/ldec)
def esc_prob(ldec,L_esc_min,L_esc_max):
    return ldec/(L_esc_max-L_esc_min)*(math.exp(-L_esc_min/ldec)-math.exp(-L_esc_max/ldec))

In [6]:
pdg_dic = {"pi0" : "111", "eta" : "221"}
FASER1_ACCEPT=0.0003
#Flipping all the negative momenta to positive momenta.
def Read_HEPEVT(file,cutoff):
    with open(file) as ef:
        num_trials=0
        pi0=[]
        eta=[]
        for line in ef:
            line = line.split()
            if len(line)==3 and line[0]=='C':
                num_trials+=1
            if len(line)==13 and line[2]==pdg_dic['pi0'] and float(line[6])>cutoff:
                pi0+=[[float(line[i]) for i in range(3,7)]]
            if len(line)==13 and line[2]==pdg_dic['eta'] and float(line[6])>cutoff:
                eta+=[[float(line[i]) for i in range(3,7)]]
    for i in range(0,len(pi0)):
        pi0[i][2]=abs(pi0[i][2])
    for i in range(0,len(eta)):
        eta[i][2]=abs(eta[i][2])
    return 2*num_trials,pi0,eta
def Build_Particle_File(arr,outfile):
    arr=[line+[0,0,0,0] for line in arr]
    np.savetxt(outfile,arr,delimiter=' ')

In [7]:
def Read_HEPEVT_MATHUSLA(file,cutoff):
    with open(file) as ef:
        num_trials=0
        pi0=[]
        eta=[]
        for line in ef:
            line = line.split()
            if len(line)==3 and line[0]=='C':
                num_trials+=1
            if len(line)==13 and line[2]==pdg_dic['pi0'] and float(line[6])>cutoff:
                pi0+=[[float(line[i]) for i in range(3,7)]]
            if len(line)==13 and line[2]==pdg_dic['eta'] and float(line[6])>cutoff:
                eta+=[[float(line[i]) for i in range(3,7)]]
        for i in range(0,len(pi0)):
            pi0[i][2]=abs(pi0[i][2])
            pi0[i][1]=abs(pi0[i][1])
        for i in range(0,len(eta)):
            eta[i][2]=abs(eta[i][2])
            eta[i][1]=abs(eta[i][1])
    return 4*num_trials,pi0,eta

In [8]:
lephold=list();alephold=list(); dphold=list();
def Event_Parser(event_file, cut):
    with open(event_file) as ef:
        dat=ef.read().splitlines()
    dat2 = [line.split() for line in dat]
    sum_line= dat2[-1]
    #return dat2
    weight_tab=[]
    dp_tab=[]
    lep=[]
    antilep=[]
    for i in range(len(dat2)):
        if len(dat2[i])==3 and float(dat2[i][2])>0:
            weight_tab+=[float(dat2[i][2])]
            booldp=False;boollep=False;boolalep=False;
            for j in range(i,len(dat2)):
                if len(dat2[j])<=1:
                    break
                if dat2[j][0]=="Dark_Photon":
                    dp_tab+=[[float(dat2[j][k]) for k in range(1,len(dat2[j]))]]
                    booldp=True
                elif dat2[j][0]=="Decay_Electron" or dat2[j][0]=="Decay_Muon":
                    lep+=[[float(dat2[j][k]) for k in range(1,len(dat2[j]))]+[dat2[j][0]]]
                    boollep=True
                elif dat2[j][0]=="Decay_Positron" or dat2[j][0]=="Decay_Antimuon":
                    antilep+=[[float(dat2[j][k]) for k in range(1,len(dat2[j]))]+[dat2[j][0]]]
                    boolalep=True
            if booldp and boollep and boolalep:
                continue
            else:
                print("PROBLEM LINE {}".format(i))
    #weight_tab=[float(line[2]) for line in dat2 if len(line)==3 and line[0]=="event" and float(line[2])>0]
    #dp_tab=[[float(line[i]) for i in range(1,len(line))] for line in dat2 if len(line)>1\
    #            and line[0]=='Dark_Photon']
    #lep = [[float(line[i]) for i in range(1,len(line))]+[line[0]] for line in dat2 if len(line)>1\
    #            and (line[0]=='Decay_Electron' or line[0]=='Decay_Muon')]
    #antilep = [[float(line[i]) for i in range(1,len(line))]+[line[0]] for line in dat2 if len(line)>1 and (line[0]=='Decay_Positron' or line[0]=='Decay_Antimuon')]
    mv = sum_line[3]
    ma = sum_line[2]
    gagpg=float(sum_line[6])
    nevents = float(sum_line[1])
    POT = float(sum_line[10])
    dp_lifetime=hbar/float(sum_line[8])
    eff=float(sum_line[11])
    #global lephold, alephold,dphold
    #lephold=lep; alephold=antilep; dphold=dp_tab;
    nevents,weight_tab,dp_tab,lep_tab,antilep_tab=cut(nevents,weight_tab,dp_tab,lep,antilep)
    return [float(mv),float(ma),gagpg,eff,dp_lifetime,nevents,weight_tab,dp_tab,lep,antilep]

In [9]:
def dec_prob(l1,l2):
    return math.exp(-l1)-math.exp(-l2)
def dec_loc(l1,l2):
    try:
        return -math.log(math.exp(-l1)-random.random()*(math.exp(-l1)-math.exp(-l2)))
    except:
        return l1
def solve_dec_loc(l1,l2,pos):
    return (10**pos-math.exp(-l1))/(math.exp(-l2)-math.exp(-l1))
def dec_loc_set(l1,l2,pos):
    try:
        return -math.log(math.exp(-l1)-pos*(math.exp(-l1)-math.exp(-l2)))
    except:
        return l1
def Return_True(dat,dp,lep,alep,mv,l):
    return True
def Rescale_Events(dat,dp,lep,alep,mv,lifetime,tot_weight,g,gset,events,cut,state=None,BR=1):
    if state is not None:
        random.setstate(state)
    #global weight_compare
    prob=0
    life=lifetime*(gset/g)**2
    #print(life,gset,g)
    l=1.0/speed_of_light/life*mv
    #print(l)
    for i,u in enumerate(dat):
        ptmp=dec_prob(u[1]*l/u[0],u[2]*l/u[0])
        if cut(u,dp[i],lep[i],alep[i],mv,l):
            #weight_compare+=[ptmp,u]
            prob+=ptmp
        else:
            continue
    return prob/tot_weight*events*(g/gset)**2*BR

In [119]:
#mv,ma,gagpg_eval,eff,lifetime,nevents,weight_tab,dp_dat,lep,alep=calc_params('Events_FASER/FASER1_decay_0.05_0.0031.dat',2.3,FASER1_Cut)

In [10]:
def calc_params(file,evnt,cut,cut2=Return_True):
    #try:
    mv,ma,gagpg_eval,eff,lifetime,nevents,weight_tab,dp_dat,lep,alep=Event_Parser(file,cut)
    #return mv,ma,gagpg_eval,eff,lifetime,nevents,weight_tab,dp_dat,lep,alep
    #except:
    #return [-1,-1,-1]
    if nevents==0:
        return [-1,-1,-1]
    dat = [[mom(u),u[14],u[15]] for u in dp_dat]
    tot_weight=0
    for i in weight_tab:
        tot_weight+=i
    state=random.getstate()
    #Calculate any branching ratio effects and suppress the overall event rate by that.
    BR=nevents/Rescale_Events(dat,dp_dat,lep,alep,mv,lifetime,tot_weight,gagpg_eval,gagpg_eval,nevents,Return_True,state=state,BR=1)
    evnt_rate = lambda g : -1.0*Rescale_Events(dat,dp_dat,lep,alep,mv,lifetime,tot_weight,10**g,gagpg_eval,nevents,cut2,state=state,BR=BR)
    diff_from_evnt = lambda g : abs(evnt_rate(g)+evnt)
    diff_from_evnt_2 = lambda g : evnt-Rescale_Events(dat,dp_dat,lep,alep,mv,lifetime,tot_weight,g,gagpg_eval,nevents,cut2,state=state,BR=BR)
    opt=minimize_scalar(evnt_rate,bounds=(-6,2),method='bounded')
    #print([evnt_rate(math.log10(0.001)),evnt_rate(math.log10(0.0008)),evnt_rate(math.log10(0.0006))])
    print([mv,opt.fun,10**opt.x])
    if opt.fun>-evnt:
        return [mv,-1,-1]
  
    opt2=minimize_scalar(diff_from_evnt,bounds=(-8,opt.x),method='bounded')

    x=opt.x
    xstep=0.1
    while diff_from_evnt_2(10**x)<0:
        x+=xstep
    
    sol = root_scalar(diff_from_evnt_2, bracket=[10**(x-xstep), 10**x], method='brentq')    
    print([10**opt2.x,sol.root])
    return [mv,10**opt2.x,sol.root]

In [11]:
NA62_energy_cut=3
NA62_efficiency=1
NA62_evnt=2.3
NA62_KLR=[0,0,217]
NA62_n=[0,0,1]
NA62_pot_suppression=1.3e16/1e18
def NA62_Cut(nevents,weight_tab,dp_tab,lep_tab,antilep_tab):
    pass_index=[]
    for i in range(len(weight_tab)):
        energy = lep_tab[i][0]+antilep_tab[i][0]
        if energy>NA62_energy_cut:
            pointl,Rl=plane_cross(momvec(lep_tab[i]),startvec(lep_tab[i]),NA62_KLR,NA62_n)
            pointa,Ra=plane_cross(momvec(antilep_tab[i]),startvec(antilep_tab[i]),NA62_KLR,NA62_n)
            #print(Ra,Rl,pointa,pointl,veclen(sub(pointl,pointa)))
            if Ra>0.15 and Rl>0.15 and Ra<1 and Rl<1 and veclen(sub(pointl,pointa))>0.1:
                pass_index.append(i)
    nevents*=len(pass_index)/len(weight_tab)*NA62_efficiency
    #print(len(pass_index),len(weight_tab))
    weight_tab = [weight_tab[i] for i in pass_index]
    dp_tab = [dp_tab[i] for i in pass_index]
    lep_tab = [lep_tab[i] for i in pass_index]
    antilep_tab = [antilep_tab[i] for i in pass_index]
    return nevents,weight_tab,dp_tab,lep_tab,antilep_tab

In [12]:
MATHUSLA_energy_cut=1
MATHUSLA_efficiency=1
MATHUSLA_evnt=4
MATHUSLA_det=[0,120,200]
MATHUSLA_n=[0,1,0]
def Mathusla_Cut(nevents,weight_tab,dp_tab,lep_tab,antilep_tab):
    pass_index=[]
    for i in range(len(weight_tab)):
        if lep_tab[i][0]>MATHUSLA_energy_cut and antilep_tab[i][0]>MATHUSLA_energy_cut:
            pass_index.append(i)
    nevents*=len(pass_index)/len(weight_tab)*MATHUSLA_efficiency
    weight_tab = [weight_tab[i] for i in pass_index]
    dp_tab = [dp_tab[i] for i in pass_index]
    lep_tab = [lep_tab[i] for i in pass_index]
    antilep_tab = [antilep_tab[i] for i in pass_index]
    return nevents,weight_tab,dp_tab,lep_tab,antilep_tab

In [13]:
def Mathusla_Cut_2(dat,dp,lep,alep,mdp,l):
    loc_scaling=dec_loc(dat[1]*l/dat[0],dat[2]*l/dat[0])*l/mom(dp)
    decay_point=[dp[1+i]*loc_scaling+dp[5+i] for i in range(3)]
    lmomvec=momvec(lep);amomvec=momvec(alep);
    pointl,Rl=plane_cross(lmomvec,decay_point,MATHUSLA_det,MATHUSLA_n)
    if abs(pointl[2]-200)>100 or abs(pointl[0])>100:
        return False
    pointa,Ra=plane_cross(amomvec,decay_point,MATHUSLA_det,MATHUSLA_n)
    if abs(pointa[2]-200)>100 or abs(pointa[0])>100:
        return False
    if veclen(sub(pointl,pointa))>0.01:
        return True
    return False

In [14]:
FASER1_pos=[0,0,480.75]
FASER1_n=[0,0,1]
FASER1_radius=0.1
FASER2_pos=[0,0,482.5]
FASER2_n=[0,0,1]
FASER2_radius=1
FASER_energy_cut=100
FASER_evnt=3
def FASER1_Cut(nevents,weight_tab,dp_tab,lep_tab,antilep_tab):
    pass_index=[]
    for i in range(len(weight_tab)):
        energy = lep_tab[i][0]+antilep_tab[i][0]
        if energy>FASER_energy_cut:
            pointl,Rl=plane_cross(momvec(lep_tab[i]),startvec(lep_tab[i]),FASER1_pos,FASER1_n)
            pointa,Ra=plane_cross(momvec(antilep_tab[i]),startvec(antilep_tab[i]),FASER1_pos,FASER1_n)
            #print(Ra,Rl,pointa,pointl,veclen(sub(pointl,pointa)))
            if Ra<FASER1_radius and Rl<FASER1_radius:
                pass_index.append(i)
    nevents*=len(pass_index)/len(weight_tab)
    #print(len(pass_index),len(weight_tab))
    weight_tab = [weight_tab[i] for i in pass_index]
    dp_tab = [dp_tab[i] for i in pass_index]
    lep_tab = [lep_tab[i] for i in pass_index]
    antilep_tab = [antilep_tab[i] for i in pass_index]
    return nevents,weight_tab,dp_tab,lep_tab,antilep_tab

def FASER2_Cut(nevents,weight_tab,dp_tab,lep_tab,antilep_tab):
    pass_index=[]
    for i in range(len(weight_tab)):
        energy = lep_tab[i][0]+antilep_tab[i][0]
        if energy>FASER_energy_cut:
            pointl,Rl=plane_cross(momvec(lep_tab[i]),startvec(lep_tab[i]),FASER2_pos,FASER2_n)
            pointa,Ra=plane_cross(momvec(antilep_tab[i]),startvec(antilep_tab[i]),FASER2_pos,FASER2_n)
            #print(Ra,Rl,pointa,pointl,veclen(sub(pointl,pointa)))
            if Ra<FASER2_radius and Rl<FASER2_radius:
                pass_index.append(i)
    nevents*=len(pass_index)/len(weight_tab)
    #print(len(pass_index),len(weight_tab))
    weight_tab = [weight_tab[i] for i in pass_index]
    dp_tab = [dp_tab[i] for i in pass_index]
    lep_tab = [lep_tab[i] for i in pass_index]
    antilep_tab = [antilep_tab[i] for i in pass_index]
    return nevents,weight_tab,dp_tab,lep_tab,antilep_tab

In [15]:
def run_output_file(file,outfile,out_prepend,samplesize,mdp_in,gagpg_in):
    with open(file) as f:
        b=f.readlines()
    mdp=str(mdp_in)
    gagpg=str(gagpg_in)
    rep_arr=[["dark_photon_mass",mdp],["samplesize",samplesize],["gagpg",gagpg],["output_file",out_prepend+"_{}_{}.dat".format(gagpg,mdp)]]
    b=update_param(b,rep_arr)
    with open(outfile,'w') as f:
            f.writelines(b)
    subp.call(["./build/main", outfile])

In [20]:
massarr=[0.00175,0.0018,0.0019,0.002,0.0025,0.00255,0.00173,0.00257]
for i in massarr:
    run_output_file("adp_cards/run_FASER1_adp.dat","adp_cards/run_FASER1_adp_2.dat","Events_FASER/FASER1_decay",150000,i,0.05)

In [22]:
files=glob.glob('Events_FASER/FASER1_decay*')
bounds=[calc_params(file,FASER_evnt,FASER1_Cut) for file in files]
bounds = [x for x in bounds if x[0]!=-1 and (x[1]!=-1 and x[2]!=-1)]
bounds.sort()
record_list("ADP_FASER1.dat",bounds)

[0.002, -3.590655771789014, 0.12358322018099589]
[0.08729061252616759, 0.1693751934842883]
[0.0025, -3.1733876615991714, 0.0792519860180304]
[0.06569031968481165, 0.0947092926691939]
[0.00175, -3.1098744313865954, 0.16107567103381348]
[0.1383382090815219, 0.1862327084408939]
[0.00255, -3.093818382697623, 0.07654279268588245]
[0.06667455449817196, 0.08740868045146616]
[0.0019, -3.4771321832110775, 0.1370234341415686]
[0.10005320154225547, 0.1825137268090554]
[0.00173, -3.048710579304782, 0.16520321924735054]
[0.14932938045145552, 0.1821846849592358]
[0.00257, -3.0488192820850175, 0.0750975516261796]
[0.06800540708581478, 0.08269586698744841]
[0.0018, -3.270067092463289, 0.1524168403330173]
[0.120153399459085, 0.1901614194258637]


In [24]:
massarr=[0.0011,0.00115,0.0012,0.0013,0.0014,0.0015,0.00175,0.002,0.003,0.004,0.005,0.006,0.008,0.01,0.015,0.02,0.025,0.026,0.027,2.72/1000.0,2.74/1000.0,2.76/1000.0]
for i in massarr:
    run_output_file("adp_cards/run_FASER2_adp.dat","adp_cards/run_FASER2_adp_2.dat","Events_FASER/FASER2_decay",150000,i,0.001)

In [25]:
files=glob.glob('Events_FASER/FASER2_decay*')
bounds=[calc_params(file,FASER_evnt,FASER2_Cut) for file in files]
bounds = [x for x in bounds if x[0]!=-1 and (x[1]!=-1 and x[2]!=-1)]
bounds.sort()
record_list("ADP_FASER2.dat",bounds)

[0.0011, -17.902972762232654, 0.2534264630744699]
[0.08594038819030869, 0.7619638554603448]
[0.002, -10504.374928832887, 0.08158709646945937]
[0.005162000235921683, 0.5464966342983589]
[0.004, -4428.600683767754, 0.021595102769438844]
[0.0016642811167306077, 0.1273170029539456]
[0.015, -45.22726417909236, 0.0015817871072782133]
[0.00040461370458696333, 0.005354003912348446]
[0.00115, -114.93579357352851, 0.23286410076866976]
[0.04602782182260705, 1.0230608997218185]
[0.006, -1456.7898049671662, 0.009724805740692172]
[0.0009897677621069916, 0.0516266606239389]
[0.00175, -8610.613709769663, 0.10467811137308511]
[0.006989982431046332, 0.7051218382325659]
[0.0015, -4745.1971285029595, 0.14092583465491268]
[0.010825128600819965, 0.9122873726322324]
[0.0012, -354.9064462310749, 0.21371861337762804]
[0.031536884840199675, 1.106808635828282]
[0.027, -3.4316356844761238, 0.0004942688550436142]
[0.00036974207473586566, 0.0006617888711141337]
[0.008, -575.2906092633448, 0.00549894611576473]
[0.00

In [13]:
massarr=[1.05,1.1,1.2,1.5,2,3,5,7.5,10,15,20,30,40,50,60,70,80,90,100,105,106,107,108,109]
for i in massarr:
    run_output_file("run_NA62.dat","run_NA62_b.dat","events_na62_decay/na62_decay",str(1e5),i/1000.0,1e-4)

In [105]:
files=glob.glob('events_na62_decay/na62_decay*')
bounds=[calc_params(file,NA62_evnt,NA62_Cut) for file in files]
bounds = [x for x in bounds if x[0]!=-1 and (x[1]!=-1 and x[2]!=-1)]
bounds.sort()
record_list("ADP_NA62.dat",bounds)

NameError: name 'record_list' is not defined

In [50]:
part_arr_list=list()
weight_tab=list()
for i in range(len(dat2)):
    if len(dat2[i])==3:
        part_arr=[]
        for j in range(i,len(dat2)):
            if len(dat2[j])<=1:
                break
            if dat2[j][0]=="Dark_Photon" or dat2[j][0]=="Decay_Electron" or dat2[j][0]=="Decay_Positron":
                part_arr+=[dat2[j]]
        part_arr_list+=[part_arr]

In [7]:
FASER_num_trials,pi0,eta=Read_HEPEVT("14tev_EPOS.dat",100)

In [8]:
FASER_num_trials,pi0_nocut,eta_nocut=Read_HEPEVT("14tev_EPOS.dat",0)

In [49]:
FASER_num_trials,pi0_sibyll,eta_sibyll=Read_HEPEVT("14tev.dat",100)

In [52]:
FASER1_pi0_sibyll=[mom4 for mom4 in pi0_sibyll if theta(mom4)<FASER1_ACCEPT]
FASER1_eta_sibyll=[mom4 for mom4 in eta_sibyll if theta(mom4)<FASER1_ACCEPT]

In [10]:
FASER1_ACCEPT=0.0003
FASER2_ACCEPT=0.003
FASER1_pi0=[mom4 for mom4 in pi0 if theta(mom4)<FASER1_ACCEPT]
FASER1_eta=[mom4 for mom4 in eta if theta(mom4)<FASER1_ACCEPT]
FASER2_pi0=[mom4 for mom4 in pi0 if theta(mom4)<FASER2_ACCEPT]
FASER2_eta=[mom4 for mom4 in eta if theta(mom4)<FASER2_ACCEPT]

In [14]:
Build_Particle_File(FASER1_pi0,"data/FASER1_pi0.dat")
Build_Particle_File(FASER1_eta,"data/FASER1_eta.dat")

In [24]:
MATHUSLA_num_trials,pi0,eta=Read_HEPEVT_MATHUSLA("14tev_EPOS.dat",2)

In [105]:
pie=[float(pi[3]) for pi in pi0 if pi[1]/pi[2]>1.0/5.0]

In [107]:
np.average(pie)

3.381906262503533

In [27]:
Build_Particle_File(pi0,"data/MATHUSLA_pi0.dat")
Build_Particle_File(eta,"data/MATHUSLA_eta.dat")

In [26]:
massarr=[1.5,2,3,1.35,1.4,1.45,1.75,2.5,3.1,3.2,3.3,3.4,3.5,3.6,3.7,3.8,3.9]
for i in massarr:
    run_output_file("adp_cards/run_mathusla_adp.dat","adp_cards/run_mathusla_adp_2.dat","Events_Math/Mathusla_decay",str(1e5),i/1000.0,1e-3)

In [27]:
files=glob.glob('Events_Math/Mathusla_decay*')
bounds=[calc_params(file,MATHUSLA_evnt,Mathusla_Cut,cut2=Mathusla_Cut_2) for file in files]
bounds = [x for x in bounds if x[0]!=-1 and (x[1]!=-1 and x[2]!=-1)]
bounds.sort()
record_list("ADP_Mathusla.dat",bounds)

[0.0035, -5.0841837855758065, 0.004130770690382302]
[0.0030313123427889834, 0.0055375229443036696]
[0.0036, -4.830736498854999, 0.003922836081938445]
[0.0029822777094410395, 0.005090107048600203]
[0.0025, -11.249994780947127, 0.008024481536452088]
[0.004065690960043305, 0.01458642096691461]
[0.0037, -4.2482853243249155, 0.0036800588501843087]
[0.003163453806451316, 0.004266459296601253]
[0.00145, -8.247664720583442, 0.022427875349697705]
[0.01300900162860051, 0.0361051790792043]
[0.003, -7.651910078704525, 0.005623733447706097]
[0.0033188041386981795, 0.009058070384091595]
[0.0014, -6.575126755335517, 0.02392200286292111]
[0.0153607477560649, 0.03559355235402559]
[0.0031, -7.1756478275792, 0.0052519565933021265]
[0.0031927874309211307, 0.00827284605242584]
[0.00135, -4.979619650968005, 0.0254534389540839]
[0.019134832260693336, 0.03318596405881574]
[0.0034, -5.665429729291609, 0.00439399512463395]
[0.0030105219682310326, 0.006244641394484106]
[0.0033, -6.040655393796661, 0.004646367982

In [ ]:
FASER1_pos=[0,0,480.75]
FASER1_n=[0,0,1]
FASER1_radius=0.1
FASER2_pos=[0,0,482.5]
FASER2_n=[0,0,1]
FASER2_radius=1
def ADP_DP_decay_calc_FASER(event_file,summary_file,radius=FASER1_radius,pos=FASER1_pos,n=FASER1_n):
    with open(event_file) as ef:
        dat=ef.read().splitlines()
        run_num=''
        for line in dat:
            line=line.split()
            if len(line)==2 and line[0]=="Run":
                run_num=line[1]
                break
    #print(event_file)
    print("Searching for Run " + run_num)
    mdp,ma,Gaggp,nevents = find_summary(summary_file,run_num)
    if nevents == -1:
        print("No events to catalogue.")
        return [-1,-1]
    
    print("Run located, " + str(nevents) + " events.")
    dat2 = [line.split() for line in dat]
    electron = [[float(line[i]) for i in range(1,len(line))] for line in dat2 if len(line)>1 and line[0]=='Decay_Electron']
    positron = [[float(line[i]) for i in range(1,len(line))] for line in dat2 if len(line)>1 and line[0]=='Decay_Positron']
    if len(electron) != len(positron):
        print("ELECTRONS AND POSITRONS DON'T MATCH!")
        return -1
    count=0;
    for i in range(len(electron)):
        if electron[i][3]+positron[i][3]>100 and electron[i][2]>0 and positron[i][2]>0 and \
        plane_cross(electron[i][0:3],electron[i][4:7],pos,n)<radius and \
        plane_cross(positron[i][0:3],electron[i][4:7],pos,n)<radius:
            count+=1
    print(float(count)/float(len(electron)),len(electron),count)
    return([mdp,gagpg,nevents*count/len(electron)])
def calc_event_rate_FASER(file,mdp_in,gagpg_in):
    mdp=str(mdp_in)
    gagpg=str(gagpg_in)
    rep_arr=[["dark_photon_mass",mdp],["samplesize",str(5000)],["gagpg",gagpg],["output_file","Events_FASER/FASER1_{}_{}.dat".format(gagpg,mdp)]]
    file=update_param(file,rep_arr)
    with open("run_FASER1_adp_2.dat",'w') as f:
            f.writelines(file)
    subp.call(["./build/main", "run_FASER1_adp_2.dat"])
    m,g,e=ADP_DP_decay_calc_FASER("Events_FASER/FASER1_{}_{}.dat".format(gagpg,mdp),"Events_FASER/FASER1.dat")
    return(e)

In [15]:
Event_Cut=2.3
gagpg_start=0.05
data_arr=[]
Iteration_Ratio=1/1.05
with open("run_FASER1_adp.dat",'r') as f:
    b=f.readlines()
#dpmarr=[3.1,3.2,3.3,3.4,3.5,3.6,3.7,3.8,3.9]
#dpmarr=[,4,5,6,7,8,9,10,11,12]
dpmarr=[3.325,3.35]
#dpmarr=[33.5]
#dpmarr.reverse()
with open("FASER1_top_curve.dat",'a') as backup:
    for mdp in dpmarr:
        random.shuffle(FASER2_pi0)
        random.shuffle(FASER2_eta)
        Build_Particle_File(FASER2_pi0,"data/FASER1_pi0.dat")
        Build_Particle_File(FASER2_eta,"data/FASER1_eta.dat")
        gagpg=gagpg_start
        f = lambda x : calc_event_rate_FASER(b,mdp/1000.0,x)
        above = False
        below = False
        below_g=0; below_e=0
        above_g=1; above_e=0
        prev_step=0
        tmp_arr=[]
        while not above or not below:
            print("Testing {} {} MeV next".format(gagpg,mdp))
            event_estimate = f(gagpg)
            backup.write("{} {} {}\n".format(mdp,gagpg,event_estimate))
            if event_estimate>Event_Cut:
                print("Above!",event_estimate)
                if above and prev_step<event_estimate:
                    print("End of contour encountered.")
                    break
                above=True
                above_g=gagpg
                above_e=event_estimate
                tmp_arr = tmp_arr + [mdp,gagpg,event_estimate]
                gagpg=gagpg/Iteration_Ratio
           
            elif event_estimate<Event_Cut:
                print("Below!",event_estimate)
                if below and prev_step>event_estimate:
                    print("End of contour encountered.")
                    break
                below=True
                below_g=gagpg
                below_e=event_estimate
                tmp_arr = tmp_arr + [mdp,gagpg,event_estimate]
                gagpg=gagpg*Iteration_Ratio
         
            if gagpg<1e-7:
                print("No events found! Exceeded gagpg limits!")
                break

            prev_step=event_estimate

        if above and below:
            mid_g=(below_g+above_g)/2.0
            print("Midpoint Test of {}".format(mid_g))
            mid_e=f(mid_g)
            backup.write("{} {} {}\n".format(mdp,mid_g,mid_e))
            tmp_arr = tmp_arr + [mdp,mid_g,mid_e]   
            #gagpg_start=below_g
            gagpg_start=above_g
        else:
            break
            
        data_arr=data_arr+[tmp_arr]

Testing 0.05 3.325 MeV next
Searching for Run 1552869573
Run located, 2.30454 events.
0.9948 5000 4974
Below! 2.292556392
Testing 0.047619047619047616 3.325 MeV next
Searching for Run 1552869786
Run located, 2.36685 events.
0.9958 5000 4979
Above! 2.35690923
Midpoint Test of 0.04880952380952381
Searching for Run 1552869996
Run located, 2.30207 events.
0.995 5000 4975
Testing 0.047619047619047616 3.35 MeV next
Searching for Run 1552870212
Run located, 2.25213 events.
0.9924 5000 4962
Below! 2.2350138120000005
Testing 0.04535147392290249 3.35 MeV next
Searching for Run 1552870456
Run located, 2.33023 events.
0.99 5000 4950
Above! 2.3069276999999997
Midpoint Test of 0.04648526077097505
Searching for Run 1552870698
Run located, 2.26587 events.
0.9944 5000 4972


In [16]:
def ADP_decay_calc(event_file,summary_file,event_cut,L_esc_min,L_esc_max,tol=1e-2):
    with open(event_file) as ef:
        dat=ef.read().splitlines()
        run_num=''
    
        for line in dat:
            line=line.split()
            if len(line)==2 and line[0]=="Run":
                run_num=line[1]
                break
    #print(event_file)
    print("Searching for Run " + run_num)
    mdp,ma,Gaggp,nevents = find_summary(summary_file,run_num)
    if nevents == -1:
        print("No events to catalogue.")
        return [-1,-1]
    
    print("Run located, " + str(nevents) + " events.")
    dat2 = [line.split() for line in dat]
    axion = [[float(line[i]) for i in range(1,len(line))] for line in dat2 if len(line)>1 and line[0]=='Recoil_Axion']
    DP = [[float(line[i]) for i in range(1,len(line))] for line in dat2 if len(line)>1 and line[0]=='Recoil_Dark_Photon']
    DPLEN=len(DP)
    AXIONLEN=len(axion)
    TOTLEN=DPLEN+AXIONLEN
    DP_E=[part[3] for part in DP]

    if nevents==0:
        return [mdp,-1]
    
    nevents=nevents/(Gaggp)**4
    eff=1
    for i in range(1000):
        Gaggp = (event_cut/nevents/eff)**0.25
        Gam=Width_A_to_a_gamma(Gaggp,ma,mdp)
        esc =np.mean([esc_prob(ldec(Gam,x,mdp),L_esc_min,L_esc_max) for x in DP_E])
        eff=(AXIONLEN+DPLEN*esc)/TOTLEN
        if abs(Gaggp**4*nevents*eff-event_cut)/event_cut < tol:
            break
    print(eff)
    return([mdp,Gaggp])

In [17]:
def ADP_DP_decay_calc(event_file,summary_file):
    with open(event_file) as ef:
        dat=ef.read().splitlines()
        run_num=''
        for line in dat:
            line=line.split()
            if len(line)==2 and line[0]=="Run":
                run_num=line[1]
                break
    #print(event_file)
    print("Searching for Run " + run_num)
    mdp,ma,Gaggp,nevents = find_summary(summary_file,run_num)
    if nevents == -1:
        print("No events to catalogue.")
        return [-1,-1]
    
    print("Run located, " + str(nevents) + " events.")
    dat2 = [line.split() for line in dat]
    electron = [[float(line[i]) for i in range(1,len(line))] for line in dat2 if len(line)>1 and line[0]=='Decay_Electron']
    positron = [[float(line[i]) for i in range(1,len(line))] for line in dat2 if len(line)>1 and line[0]=='Decay_Positron']
    if len(electron) != len(positron):
        print("ELECTRONS AND POSITRONS DON'T MATCH!")
        return -1
    count=0;
    for i in range(len(electron)):
        if electron[i][3]+positron[i][3]>3:
            count+=1
    print(float(count)/float(len(electron)),len(electron),count)
    return([mdp,gagpg,nevents*count/len(electron)])

In [6]:
ship_pos=[0,0,100]
ship_n=[0,0,1]
ship_radius=2.5

In [44]:
def ADP_DP_decay_calc_SHiP(event_file,summary_file):
    with open(event_file) as ef:
        dat=ef.read().splitlines()
        run_num=''
        for line in dat:
            line=line.split()
            if len(line)==2 and line[0]=="Run":
                run_num=line[1]
                break
    #print(event_file)
    print("Searching for Run " + run_num)
    mdp,ma,Gaggp,nevents = find_summary(summary_file,run_num)
    if nevents == -1:
        print("No events to catalogue.")
        return [-1,-1]
    
    print("Run located, " + str(nevents) + " events.")
    dat2 = [line.split() for line in dat]
    electron = [[float(line[i]) for i in range(1,len(line))] for line in dat2 if len(line)>1 and line[0]=='Decay_Electron']
    positron = [[float(line[i]) for i in range(1,len(line))] for line in dat2 if len(line)>1 and line[0]=='Decay_Positron']
    if len(electron) != len(positron):
        print("ELECTRONS AND POSITRONS DON'T MATCH!")
        return -1
    count=0;
    for i in range(len(electron)):
        if electron[i][3]>1 and electron[i][2]>0 and positron[i][3]>1 and positron[i][2]>0 and plane_cross(electron[i][0:3],electron[i][4:7],ship_pos,ship_n)<ship_radius and plane_cross(positron[i][0:3],electron[i][4:7],ship_pos,ship_n)<ship_radius:
            count+=1
    print(float(count)/float(len(electron)),len(electron),count)
    return([mdp,gagpg,nevents*count/len(electron)])

In [7]:
def ADP_DP_decay_gamma_a_calc_SHiP(event_file,summary_file):
    with open(event_file) as ef:
        dat=ef.read().splitlines()
        run_num=''
        for line in dat:
            line=line.split()
            if len(line)==2 and line[0]=="Run":
                run_num=line[1]
                break
    #print(event_file)
    print("Searching for Run " + run_num)
    mdp,ma,Gagpg,nevents = find_summary(summary_file,run_num)
    if nevents == -1:
        print("No events to catalogue.")
        return [-1,-1]
    
    print("Run located, " + str(nevents) + " events.")
    dat2 = [line.split() for line in dat]
    photon = [[float(line[i]) for i in range(1,len(line))] for line in dat2 if len(line)>1 and line[0]=='Decay_Photon']
    if len(photon)==0 or nevents==0:
        print(0,0,0)
        return([mdp,Gagpg,0])
    count=0;
    for i in range(len(photon)):
        if photon[i][3]>1 and photon[i][2]>0 and plane_cross(photon[i][0:3],photon[i][4:7],ship_pos,ship_n)<ship_radius:
            count+=1
    print(float(count)/float(len(photon)),len(photon),count)
    return([mdp,Gagpg,nevents*count/len(photon)])

In [8]:
def calc_event_rate_ship_gamma_a(file,mdp_in,gagpg_in):
    mdp=str(mdp_in)
    gagpg=str(gagpg_in)
    rep_arr=[["dark_photon_mass",mdp],["samplesize",str(1000)],["gagpg",gagpg],["output_file","Events_ship_decay/ship_{}_{}.dat".format(gagpg,mdp)]]
    file=update_param(file,rep_arr)
    with open("run_ship_decay_adp_2.dat",'w') as f:
            f.writelines(file)
    subp.call(["./build/main", "run_ship_decay_adp_2.dat"])
    m,g,e=ADP_DP_decay_gamma_a_calc_SHiP("Events_ship_decay/ship_{}_{}.dat".format(gagpg,mdp),"Events_ship_decay/ship.dat")
    return(e)

In [12]:
#Try 0.008 with 20 MeV
#SHiP gamma decay
Event_Cut=10
with open("run_ship_decay_adp.dat") as f:
    b=f.readlines()
dpmarr=[270,280,290,300]
#[1,2,3,7.5,10,15,20,40,60,80,100,120,130,140,170,200,220,240,250,260]
g_start=1.7e-5
data_arr=[]
Iteration_Ratio=1.15
with open("ship_decay_curve.dat",'a') as backup:
    for mdp in dpmarr:
        climbing=True
        Searching=True
        g=g_start
        f = lambda x : calc_event_rate_ship_gamma_a(b,mdp/1000.0,x)
        prev_step=0
        tmp_arr=[]
        while True:
            if g<10**-7:
                break
            print("Testing m={} MeV g={} next.".format(mdp,g))
            event = f(g)
            print("{} events".format(event))
            backup.write("{} {} {}\n".format(mdp,g,event))
            if Searching:
                if event>Event_Cut:
                    print("Found upper edge.")
                    climbing=True
                    Searching=False
                    g_start=g*Iteration_Ratio
            elif climbing:
                if event<prev_step:
                    print("Inflection point found.")
                    climbing=False
            else:
                if event<Event_Cut:
                    break
            if event>5000:
                g=g/Iteration_Ratio**2
            g=g/Iteration_Ratio
            prev_step=event

Testing m=270 MeV g=1.7e-05 next.
Searching for Run 1550588225
Run located, 3.82451 events.
0.994 1000 994
3.80156294 events
Testing m=270 MeV g=1.4782608695652174e-05 next.
Searching for Run 1550588430
Run located, 9.14427 events.
0.995 1000 995
9.098548650000001 events
Testing m=270 MeV g=1.285444234404537e-05 next.
Searching for Run 1550588576
Run located, 18.2786 events.
0.989 1000 989
18.077535400000002 events
Found upper edge.
Testing m=270 MeV g=1.1177775951343801e-05 next.
Searching for Run 1550588701
Run located, 31.6903 events.
0.99 1000 990
31.373397 events
Testing m=270 MeV g=9.719805175081566e-06 next.
Searching for Run 1550588818
Run located, 46.6445 events.
0.984 1000 984
45.898188000000005 events
Testing m=270 MeV g=8.452004500070928e-06 next.
Searching for Run 1550588927
Run located, 63.4886 events.
0.975 1000 975
61.901385 events
Testing m=270 MeV g=7.349569130496459e-06 next.
Searching for Run 1550589034
Run located, 76.7442 events.
0.98 1000 980
75.209316 events
Tes

Searching for Run 1550594580
Run located, 18.6415 events.
0.984 1000 984
18.343236 events
Testing m=300 MeV g=6.3909296786925735e-06 next.
Searching for Run 1550594688
Run located, 25.9294 events.
0.983 1000 983
25.4886002 events
Testing m=300 MeV g=5.557330155384847e-06 next.
Searching for Run 1550594795
Run located, 29.2515 events.
0.964 1000 964
28.198446 events
Testing m=300 MeV g=4.832461004682476e-06 next.
Searching for Run 1550594900
Run located, 31.9306 events.
0.957 1000 957
30.557584199999997 events
Testing m=300 MeV g=4.202140004071719e-06 next.
Searching for Run 1550595005
Run located, 33.4526 events.
0.939 1000 939
31.411991399999994 events
Testing m=300 MeV g=3.654034786149321e-06 next.
Searching for Run 1550595109
Run located, 32.6014 events.
0.917 1000 917
29.895483799999997 events
Inflection point found.
Testing m=300 MeV g=3.177421553173323e-06 next.
Searching for Run 1550595212
Run located, 30.6579 events.
0.88 1000 880
26.978952 events
Testing m=300 MeV g=2.76297526

In [10]:
def calc_event_rate_ship(file,mdp_in,gagpg_in):
    mdp=str(mdp_in)
    gagpg=str(gagpg_in)
    rep_arr=[["dark_photon_mass",mdp],["samplesize",str(1000)],["gagpg",gagpg],["output_file","Events_ship_decay/ship_{}_{}.dat".format(gagpg,mdp)]]
    file=update_param(file,rep_arr)
    with open("run_ship_decay_adp_2.dat",'w') as f:
            f.writelines(file)
    subp.call(["./build/main", "run_ship_decay_adp_2.dat"])
    m,g,e=ADP_DP_decay_calc_SHiP("Events_ship_decay/ship_{}_{}.dat".format(gagpg,mdp),"Events_ship_decay/ship.dat")
    print(m,g,e)
    return(e)

In [34]:
lsnd_en_max=0.05
lsnd_en_min=0.018
lsnd_ang_res=12*math.pi/180

In [33]:
def ADP_DP_decay_calc_LSND(event_file,summary_file):
    with open(event_file) as ef:
        dat=ef.read().splitlines()
        run_num=''
        for line in dat:
            line=line.split()
            if len(line)==2 and line[0]=="Run":
                run_num=line[1]
                break
    #print(event_file)
    print("Searching for Run " + run_num)
    mdp,ma,Gaggp,nevents = find_summary(summary_file,run_num)
    if nevents == -1:
        print("No events to catalogue.")
        return [-1,-1]
    
    print("Run located, " + str(nevents) + " events.")
    dat2 = [line.split() for line in dat]
    electron = [[float(line[i]) for i in range(1,len(line))] for line in dat2 if len(line)>1 and line[0]=='Decay_Electron']
    positron = [[float(line[i]) for i in range(1,len(line))] for line in dat2 if len(line)>1 and line[0]=='Decay_Positron']
    if len(electron) != len(positron):
        print("ELECTRONS AND POSITRONS DON'T MATCH!")
        return -1
    count=0;
    for i in range(len(electron)):
        en_tot=electron[i][3]+positron[i][3]
        #print(en_tot,theta_dif(electron[i],positron[i]),cos_theta(electron[i]),cos_theta(positron[i]))
        #print(en_tot>lsnd_en_min,en_tot<lsnd_en_max,theta_dif(electron[i],positron[i])<lsnd_ang_res)
        #and (cos_theta(electron[i])<0.9 or cos_theta(positron[i])<0.9)
        if en_tot>lsnd_en_min and en_tot<lsnd_en_max and theta_dif(electron[i],positron[i])<lsnd_ang_res:
            count+=1
    print(float(count)/float(len(electron)),len(electron),count)
    return([mdp,gagpg,nevents*count/len(electron)])

In [36]:
def calc_event_rate_lsnd(file,mdp_in,gagpg_in):
    mdp=str(mdp_in)
    gagpg=str(gagpg_in)
    rep_arr=[["dark_photon_mass",mdp],["samplesize",str(2000)],["gagpg",gagpg],["output_file","Events_lsnd_decay/lsnd_{}_{}.dat".format(gagpg,mdp)]]
    file=update_param(file,rep_arr)
    with open("run_lsnd_decay_adp_2.dat",'w') as f:
            f.writelines(file)
    subp.call(["./build/main", "run_lsnd_decay_adp_2.dat"])
    m,g,e=ADP_DP_decay_calc_LSND("Events_lsnd_decay/lsnd_{}_{}.dat".format(gagpg,mdp),"Events_lsnd_decay/lsnd.dat")
    print(m,g,e)
    return(e)

In [37]:
with open("run_lsnd_decay_adp.dat",'r') as f:
    b=f.readlines()
#dpmarr=[253]
dpmarr=[3,7,10,15,20,30,40,50,60,70,80,90,100]
#dpmarr=[x for x in range(241,250,1)]
#dpmarr.reverse()
gagpg_start=7e-4
data_arr=[]
#Iteration_Ratio=1.0/1.05
Iteration_Ratio=1.05
with open("lsnd_bottom_curve.dat",'a') as backup:
    for mdp in dpmarr:
        gagpg=gagpg_start
        f = lambda x : calc_event_rate_lsnd(b,mdp/1000.0,x)
        above = False
        below = False
        below_g=0; below_e=0
        above_g=1; above_e=0
        prev_step=0
        tmp_arr=[]
        while not above or not below:
            print("Testing {} next".format(gagpg))
            event_estimate = f(gagpg)
            backup.write("{} {} {}\n".format(mdp,gagpg,event_estimate))
            if event_estimate>Event_Cut:
                print("Above!",event_estimate)
                #if above and prev_step<event_estimate:
                #    print("End of contour encountered.")
                #    break
                above=True
                above_g=gagpg
                above_e=event_estimate
                tmp_arr = tmp_arr + [mdp,gagpg,event_estimate]
                gagpg=gagpg/Iteration_Ratio
           
            elif event_estimate<Event_Cut:
                print("Below!",event_estimate)
                #if below and prev_step>event_estimate:
                #    print("End of contour encountered.")
                #    break
                below=True
                below_g=gagpg
                below_e=event_estimate
                tmp_arr = tmp_arr + [mdp,gagpg,event_estimate]
                gagpg=gagpg*Iteration_Ratio
         
            if gagpg<1e-7 or gagpg>1:
                print("No events found! Exceeded gagpg limits!")
                break

            prev_step=event_estimate

        if above and below:
            mid_g=(below_g+above_g)/2.0
            print("Midpoint Test of {}".format(mid_g))
            mid_e=f(mid_g)
            backup.write("{} {} {}\n".format(mdp,mid_g,mid_e))
            tmp_arr = tmp_arr + [mdp,mid_g,mid_e]   
            #gagpg_start=below_g
            gagpg_start=above_g
        else:
            break
            
        data_arr=data_arr+[tmp_arr]

Testing 0.0003 next
Searching for Run 1545979381
Run located, 102.777 events.
0.404 2000 808
0.003 0.0003 41.521908
Below! 41.521908
Testing 0.00033 next
Searching for Run 1545979397
Run located, 142.761 events.
0.423 2000 846
0.003 0.00033 60.387903
Above! 60.387903
Midpoint Test of 0.00031499999999999996
Searching for Run 1545979414
Run located, 122.324 events.
0.408 2000 816
0.003 0.0003 49.908192
Testing 0.00033 next
Searching for Run 1545979430
Run located, 2916.39 events.
0.2225 2000 445
0.007 0.00033 648.896775
Above! 648.896775
Testing 0.0003 next
Searching for Run 1545979436
Run located, 2444.29 events.
0.2245 2000 449
0.007 0.0003 548.743105
Above! 548.743105
Testing 0.0002727272727272727 next
Searching for Run 1545979443
Run located, 2057.72 events.
0.2515 2000 503
0.007 0.0002727272727272727 517.51658
Above! 517.51658
Testing 0.0002479338842975206 next


KeyboardInterrupt: 

In [11]:
def calc_event_rate(file,mdp_in,gagpg_in):
    mdp=str(mdp_in)
    gagpg=str(gagpg_in)
    rep_arr=[["dark_photon_mass",mdp],["samplesize",str(300)],["gagpg",gagpg],["output_file","Events_CHARM_ADP_3/CHARM_{}_{}.dat".format(gagpg,mdp)]]
    file=update_param(file,rep_arr)
    with open("run_CHARM_adp_2.dat",'w') as f:
            f.writelines(file)
    subp.call(["./build/main", "run_CHARM_adp_2.dat"])
    m,g,e=ADP_DP_decay_calc("Events_CHARM_ADP_3/CHARM_{}_{}.dat".format(gagpg,mdp),"Events_CHARM_ADP_3/CHARM_3.dat")
    print(m,g,e)
    return(e)

In [17]:
Event_Cut=2.3
with open("run_CHARM_adp.dat",'r') as f:
    b=f.readlines()
dpmarr=[1.18,1.16,1.14]
#,4,5,7,10,12,13,14
#dpmarr.reverse()
gagpg_start=0.2
data_arr=[]
Iteration_Ratio=1.0/1.2
#Iteration_Ratio=1.1
with open("CHARM_Top_curve_2.dat",'a') as backup:
#with open("CHARM_Bottom_curve_2.dat",'a') as backup:
    for mdp in dpmarr:
        gagpg=gagpg_start
        f = lambda x : calc_event_rate(b,mdp/1000.0,x)
        above = False
        below = False
        below_g=0; below_e=0
        above_g=1; above_e=0
        prev_step=0
        tmp_arr=[]
        while not above or not below:
            print("Testing {} next".format(gagpg))
            event_estimate = f(gagpg)
            backup.write("{} {} {}\n".format(mdp,gagpg,event_estimate))
            if event_estimate>Event_Cut:
                print("Above!",event_estimate)
                #if above and prev_step<event_estimate:
                #    print("End of contour encountered.")
                #    break
                above=True
                above_g=gagpg
                above_e=event_estimate
                tmp_arr = tmp_arr + [mdp,gagpg,event_estimate]
                gagpg=gagpg/Iteration_Ratio
           
            elif event_estimate<Event_Cut:
                print("Below!",event_estimate)
                #if below and prev_step>event_estimate:
                #    print("End of contour encountered.")
                #    break
                below=True
                below_g=gagpg
                below_e=event_estimate
                tmp_arr = tmp_arr + [mdp,gagpg,event_estimate]
                gagpg=gagpg*Iteration_Ratio
         
            if gagpg<1e-5 or gagpg>1:
                print("No events found! Exceeded gagpg limits!")
                break

            prev_step=event_estimate

        if above and below:
            mid_g=(below_g+above_g)/2.0
            print("Midpoint Test of {}".format(mid_g))
            mid_e=f(mid_g)
            backup.write("{} {} {}\n".format(mdp,mid_g,mid_e))
            tmp_arr = tmp_arr + [mdp,mid_g,mid_e]   
            gagpg_start=below_g
            #gagpg_start=mid_g
        else:
            break
            
        data_arr=data_arr+[tmp_arr]

Testing 0.2 next
Searching for Run 1544705527
Run located, 0.130631 events.
1.0 77 77
0.00118 0.2 0.130631
Below! 0.130631
Testing 0.16666666666666669 next
Searching for Run 1544707283
Run located, 0.523477 events.
1.0 259 259
0.00118 0.16666666666666669 0.523477
Below! 0.523477
Testing 0.13888888888888892 next
Searching for Run 1544709036
Run located, 1.78207 events.
1.0 300 300
0.00118 0.13888888888888892 1.78207
Below! 1.78207
Testing 0.11574074074074077 next
Searching for Run 1544709544
Run located, 4.26405 events.
1.0 300 300
0.00118 0.11574074074074077 4.26405
Above! 4.26405
Midpoint Test of 0.12731481481481485
Searching for Run 1544709805
Run located, 2.52904 events.
1.0 300 300
0.00118 0.13888888888888892 2.52904
Testing 0.13888888888888892 next
Searching for Run 1544710236
Run located, 1.34232 events.
1.0 300 300
0.00116 0.13888888888888892 1.34232
Below! 1.34232
Testing 0.11574074074074077 next
Searching for Run 1544710675
Run located, 2.84815 events.
1.0 300 300
0.00116 0.11

In [14]:
Event_Cut=100
with open("run_ship_decay_adp.dat",'r') as f:
    b=f.readlines()
dpmarr=[230,240,250,260,265,266,267,268]
#dpmarr=[10,15,20]
#dpmarr=[x for x in range(241,250,1)]
#[,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,190,200,210,220,230,240,250]
#dpmarr.reverse()
gagpg_start=1e-6
data_arr=[]
Iteration_Ratio=1.1
#Iteration_Ratio=1.025
with open("ship_gamma_bottom_curve.dat",'a') as backup:
    for mdp in dpmarr:
        gagpg=gagpg_start
        f = lambda x : calc_event_rate_ship_gamma_a(b,mdp/1000.0,x)
        above = False
        below = False
        below_g=0; below_e=0
        above_g=1; above_e=0
        prev_step=0
        tmp_arr=[]
        while not above or not below:
            print("Testing {} {} MeV next".format(gagpg,mdp))
            event_estimate = f(gagpg)
            backup.write("{} {} {}\n".format(mdp,gagpg,event_estimate))
            if event_estimate>Event_Cut:
                print("Above!",event_estimate)
                #if above and prev_step<event_estimate:
                #    print("End of contour encountered.")
                #    break
                above=True
                above_g=gagpg
                above_e=event_estimate
                tmp_arr = tmp_arr + [mdp,gagpg,event_estimate]
                gagpg=gagpg/Iteration_Ratio
           
            elif event_estimate<Event_Cut:
                print("Below!",event_estimate)
                #if below and prev_step>event_estimate:
                #    print("End of contour encountered.")
                #    break
                below=True
                below_g=gagpg
                below_e=event_estimate
                tmp_arr = tmp_arr + [mdp,gagpg,event_estimate]
                gagpg=gagpg*Iteration_Ratio
         
            if gagpg<1e-7:
                print("No events found! Exceeded gagpg limits!")
                break

            prev_step=event_estimate

        if above and below:
            mid_g=(below_g+above_g)/2.0
            print("Midpoint Test of {}".format(mid_g))
            mid_e=f(mid_g)
            backup.write("{} {} {}\n".format(mdp,mid_g,mid_e))
            tmp_arr = tmp_arr + [mdp,mid_g,mid_e]   
            #gagpg_start=below_g
            gagpg_start=above_g
        else:
            break
            
        data_arr=data_arr+[tmp_arr]

Testing 1e-06 230 MeV next
Searching for Run 1550898043
Run located, 13.168 events.
0.475 1000 475
Below! 6.2547999999999995
Testing 1.1e-06 230 MeV next
Searching for Run 1550898258
Run located, 17.5598 events.
0.473 1000 473
Below! 8.3057854
Testing 1.21e-06 230 MeV next
Searching for Run 1550898487
Run located, 23.5916 events.
0.459 1000 459
Below! 10.8285444
Testing 1.3310000000000003e-06 230 MeV next
Searching for Run 1550898671
Run located, 32.7021 events.
0.484 1000 484
Below! 15.827816400000001
Testing 1.4641000000000003e-06 230 MeV next
Searching for Run 1550898884
Run located, 42.9343 events.
0.493 1000 493
Below! 21.1666099
Testing 1.6105100000000006e-06 230 MeV next
Searching for Run 1550899113
Run located, 56.9604 events.
0.527 1000 527
Below! 30.018130799999998
Testing 1.7715610000000007e-06 230 MeV next
Searching for Run 1550899338
Run located, 70.271 events.
0.552 1000 552
Below! 38.789592
Testing 1.948717100000001e-06 230 MeV next
Searching for Run 1550899566
Run locat

In [8]:
def ADP_DP_decay_a_gamma_calc(event_file,summary_file):
    with open(event_file) as ef:
        dat=ef.read().splitlines()
        run_num=''
        for line in dat:
            line=line.split()
            if len(line)==2 and line[0]=="Run":
                run_num=line[1]
                break
    #print(event_file)
    print("Searching for Run " + run_num)
    mdp,ma,Gaggp,nevents = find_summary(summary_file,run_num)
    if nevents == -1:
        print("No events to catalogue.")
        return [-1,-1]
    
    print("Run located, " + str(nevents) + " events.")
    dat2 = [line.split() for line in dat]
    photon = [[float(line[i]) for i in range(1,len(line))] for line in dat2 if len(line)>1 and line[0]=='Decay_Photon']
    count=0;
    for i in range(len(photon)):
        if photon[i][3]>7.5 and photon[i][3]<50 and (theta([photon[i][0],photon[i][1],photon[i][2]])*photon[i][3])**2:
            count+=1
    print(float(count)/float(len(photon)),len(photon),count)
    return([mdp,gagpg,nevents*count/len(photon)])

In [9]:
def calc_event_rate_a_gamma(file,mdp_in,gagpg_in):
    mdp=str(mdp_in)
    gagpg=str(gagpg_in)
    rep_arr=[["samplesize",str(300)],["dark_photon_mass",mdp],["gagpg",gagpg],["output_file","Events_CHARM_ADP_b/CHARM_{}_{}.dat".format(gagpg,mdp)]]
    file=update_param(file,rep_arr)
    with open("run_CHARM_adp_2b.dat",'w') as f:
            f.writelines(file)
    subp.call(["./build/main", "run_CHARM_adp_2b.dat"])
    m,g,e=ADP_DP_decay_a_gamma_calc("Events_CHARM_ADP_b/CHARM_{}_{}.dat".format(gagpg,mdp),"Events_CHARM_ADP_b/CHARM.dat")
    print(m,g,e)
    return(e)

In [57]:
def Event_Parser_a_gamma(event_file, cut, rescale):
    with open(event_file) as ef:
        dat=ef.read().splitlines()
        run_num=''
        for line in dat:
            line=line.split()
            if len(line)==2 and line[0]=="Run":
                run_num=line[1]                
                break
    dat2 = [line.split() for line in dat]
    sum_line= dat2[-1]
    weight_tab=[float(line[2]) for line in dat2 if len(line)==3 and line[0]=="event"]
    DP_tab=[[float(line[i]) for i in range(1,len(line))] for line in dat2 if len(line)>1\
                and line[0]=='Dark_Photon']
    gam_tab = [[float(line[i]) for i in range(1,len(line))]+[line[0]] for line in dat2 if len(line)>1\
                and (line[0]=='Decay_Photon')]
    ax_tab = [[float(line[i]) for i in range(1,len(line))]+[line[0]] for line in dat2 if len(line)>1\
                and (line[0]=='Decay_Axion')]
    malp = float(sum_line[2])
    mdp = float(sum_line[3])
    nevents = float(sum_line[1])*rescale
    POT = float(sum_line[10])
    gagp=float(sum_line[6])
    eps=float(sum_line[4])
    dp_lifetime=hbar/float(sum_line[8])
    eff=float(sum_line[11])
    nevents,weight_tab,DP_tab,gam_tab,ax_tab=cut(nevents,weight_tab,DP_tab,gam_tab,ax_tab)
    return [float(mdp),float(malp),gagp,eps,eff,dp_lifetime,nevents,weight_tab,DP_tab,gam_tab,ax_tab]

In [69]:
hbar

6.582e-25

In [78]:
def calc_params(file,evnt,rescale,cut,cut2=Return_True):
    #try:
    mdp,malp,gagpg_eval,eps_eval,eff,dp_lifetime,nevents,weight_tab,DP_tab,gam_tab,ax_tab=Event_Parser_a_gamma(file,cut,rescale)
    #except:
    #    print("Exception", file)
    #    return [-1,-1,-1]
    if nevents==0:
        return [-1,-1,-1]
    dat = [[mom(u),u[14],u[15]] for u in DP_tab]
    tot_weight=0
    for i in weight_tab:
        tot_weight+=i
    state=random.getstate()
    evnt_rate = lambda gagpg : -1.0*Rescale_Events_a_Gamma(dat,DP_tab,gam_tab,mdp,malp,dp_lifetime,tot_weight,10**gagpg,gagpg_eval,eps_eval,eps_eval,nevents,cut2,state)
    diff_from_evnt = lambda gagpg : abs(evnt_rate(gagpg)+evnt)
    diff_from_evnt_2 = lambda gagpg : evnt-Rescale_Events_a_Gamma(dat,DP_tab,gam_tab,mdp,malp,dp_lifetime,tot_weight,10**gagpg,gagpg_eval,eps_eval,eps_eval,nevents,cut2,state)
    opt=minimize_scalar(evnt_rate,bounds=(-6,0),method='bounded')
    if opt.fun>-evnt:
        return [mv,-1,-1]
    
    opt2=minimize_scalar(diff_from_evnt,bounds=(-8,opt.x),method='bounded')

    x=opt.x
    xstep=0.1
    while diff_from_evnt_2(10**x)<0:
        x+=xstep
    
    sol = root_scalar(diff_from_evnt_2, bracket=[10**(x-xstep), 10**x], method='brentq')    
    

    return [mx1,10**opt2.x,sol.root]

In [66]:
dat1=Event_Parser_a_gamma("Events_ADP_3/reno_adp_1mev.dat", RENO_cut,7.33639e6);

In [77]:
def Return_True(dat,DPlist,gammalist,mA,ma):
    return True
#This assumes production is through kinetic mixing
def Rescale_Events_a_Gamma(dat,DPlist,gammalist,mA,ma,lifetime,tot_weight,gagpg,gagpgset,eps,epsset,events,cut,state):
    random.setstate(state)
    prob=0
    life=lifetime*(gagpgset/gagpg)**2
    l=1.0/speed_of_light/life*mA
    for i,u in enumerate(dat):
        if i==1:
            print(u[0],u[1],u[2],life,l)
        ptmp=dec_prob(u[1]*l/u[0],u[2]*l/u[0])
        if cut(u,DPlist[i],gammalist[i],mA,ma):
            prob+=ptmp
        else:
            continue
    return prob/tot_weight*events*(eps/epsset)**2

In [24]:
min_energy_RENO=0.3*mev
def RENO_cut(nevents,weight_tab,DP_tab,gam_tab,ax_tab):
    pass_index=[]
    for i in range(len(weight_tab)):
        energy = gam_tab[i][0]
        if energy > min_energy_RENO:
            pass_index.append(i)
    weight_tab_2 = [weight_tab[i] for i in pass_index]
    DP_tab = [DP_tab[i] for i in pass_index]
    gam_tab = [gam_tab[i] for i in pass_index]
    ax_tab = [ax_tab[i] for i in pass_index]
    nevents = nevents*sum(weight_tab_2)/sum(weight_tab)
    return nevents,weight_tab_2,DP_tab,gam_tab,ax_tab

In [79]:
calc_params("Events_ADP_3/reno_adp_1mev.dat",3,7.33639e6,RENO_cut,cut2=Return_True)

0.002398296399252475 302.5640161 305.44104 5.178295724557424e-06 6.441580646239762e-07
0.002398296399252475 302.5640161 305.44104 7.609762780588879e-09 0.000438337047836778
0.002398296399252475 302.5640161 305.44104 0.0002917180176653912 1.1434470104646038e-08
0.002398296399252475 302.5640161 305.44104 1.5276128284302114e-06 2.183564375673157e-06
0.002398296399252475 302.5640161 305.44104 2.015961568766791e-07 1.6546153476636e-05
0.002398296399252475 302.5640161 305.44104 7.443393305349417e-07 4.481344482474495e-06
0.002398296399252475 302.5640161 305.44104 6.197707447397831e-07 5.382056155912979e-06
0.002398296399252475 302.5640161 305.44104 4.517990719868382e-07 7.383018600088435e-06
0.002398296399252475 302.5640161 305.44104 5.49278482938002e-07 6.072768287116792e-06
0.002398296399252475 302.5640161 305.44104 5.496722597424142e-07 6.068417848746194e-06
0.002398296399252475 302.5640161 305.44104 5.478370327709237e-07 6.088746748481146e-06
0.002398296399252475 302.5640161 305.44104 5.

ValueError: f(a) and f(b) must have different signs

In [76]:
mom(tab[0])

0.002635911051596799